In [2]:
from keras.datasets.mnist import load_data
import matplotlib.pyplot as plt
import numpy as np

from keras import Sequential
from keras.layers import Conv2D
from keras.layers import LeakyReLU
from keras.layers import Dropout
from keras.layers import Flatten
from keras.layers import Dense
from keras.layers import Reshape
from keras.layers import Conv2DTranspose
from keras.optimizers import Adam

Using TensorFlow backend.
/Users/ajoshi/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:516: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/Users/ajoshi/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:517: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/Users/ajoshi/opt/anaconda3/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:518: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/Users/ajoshi/opt/anaconda3/lib/python3.7

In [2]:
def define_discriminator(in_shape=(28,28,1)):
    model = Sequential()
    model.add(Conv2D(64, (3,3), strides=(2,2), padding="same", input_shape=in_shape))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Conv2D(64, (3,3), strides=(2,2), padding="same")) 
    model.add(LeakyReLU(alpha=0.2))
    model.add(Dropout(0.4))
    model.add(Flatten())
    model.add(Dense(1, activation="sigmoid"))
    # compile model
    adam_opt = Adam(lr=0.0002, beta_1=0.5)
    model.compile(loss='binary_crossentropy', optimizer=adam_opt, metrics=['accuracy'])
    return model

def load_and_norm_samples():
    (train_data, _), (_, _) = load_data()
    training_images = np.expand_dims(train_data, axis = -1)
    training_images = training_images.astype('float32')
    training_images = training_images / 255.0
    return training_images
    
def generate_real_images(dataset, num_samples):
    idx = np.random.randint(0, dataset.shape[0], num_samples)
    gen_data = dataset[idx]
    gen_labels = np.ones((num_samples, 1))
    return gen_data, gen_labels
    
def generate_fake_images(num_samples):
    gen_data = np.random.rand(28 * 28 * num_samples)
    gen_data = gen_data.reshape((num_samples, 28, 28, 1))
    gen_labels = np.zeros((num_samples, 1))
    return gen_data, gen_labels

def train_discriminator(model, dataset, num_periods=100, batch_size=256):
    half_batch_size = int(batch_size / 2)
    for epoch in range(num_periods):
        image_real, label_real = generate_real_images(dataset, half_batch_size)
        _, real_acc = model.train_on_batch(image_real, label_real)
        image_fake, label_fake = generate_fake_images(half_batch_size)
        _, fake_acc = model.train_on_batch(image_fake, label_fake)
        print('>%d real=%.0f%% fake=%.0f%%' % (epoch, real_acc * 100, fake_acc * 100))

In [3]:
def define_generator(latent_dim):
    model = Sequential()
    num_nodes = 128 * 7 * 7
    model.add(Dense(num_nodes, input_dim=latent_dim))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Reshape((7, 7, 128)))
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2DTranspose(128, (4,4), strides=(2,2), padding='same'))
    model.add(LeakyReLU(alpha=0.2))
    model.add(Conv2D(1, (7,7), activation='sigmoid', padding='same'))
    return model

def generate_latent_points(latent_dim, num_samples):
    train_input = np.random.randn(latent_dim * num_samples)
    train_input = train_input.reshape(num_samples, latent_dim)
    return train_input

def generate_fake_samples(g_model, latent_dim, num_samples):
    train_input = generate_latent_points(latent_dim, num_samples)
    training_images = g_model.predict(train_input)
    training_labels = np.zeros((num_samples, 1))
    return training_images, training_labels

In [ ]:
discriminator_model = define_discriminator()
dataset = load_and_norm_samples()

train_discriminator(discriminator_model, dataset)

In [4]:
latent_dim = 100
generator_model = define_generator(latent_dim)


num_samples = 25


fake_samples, _ = generate_fake_samples(generator_model, latent_dim, num_samples)

print(fake_samples.shape)



(25, 28, 28, 1)


In [ ]:
for i in range(num_samples):
    plt.subplot(5, 5, i+1)
    
    plt.axis('off')
    plt.imshow(fake_samples[i], cmap="gray_r")